In [1]:
import math
import collections
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
import utils

In [2]:
def get_price_index(year_growth_rate, month):
    return math.pow(1 + year_growth_rate/12, month)


class Simulation:
    def __init__(self, scenario, real_estate, portfolio):
        self.scenario = scenario
        self.real_estate = real_estate
        self.portfolio = portfolio

    def run(self):
        month = 0
        maturity = self.real_estate.mortgage.maturity
        n_periods = self.real_estate.mortgage.n_periods
        max_periods = maturity * n_periods
        new_monthly_payment_amount = self.real_estate.mortgage.monthly_payment + self.scenario['mortgage_overpay_month']
        self.real_estate.mortgage.update_monthly_payment_amount(new_monthly_payment_amount)
        while self.real_estate.mortgage.mortgage_amount > 0 and month <= max_periods:
            real_estate_price_index = get_price_index(self.scenario['growth_rate_real_estate'], month)
            self.real_estate.tick_month(real_estate_price_index)
            stock_price = get_price_index(self.scenario['growth_rate_stocks'], month) * self.scenario['stock_price']
            self.portfolio.purchase(self.scenario['mortgage_overpay_month'], stock_price)
            self.portfolio.update_values(stock_price)
            month += 1

In [3]:
scenario_1 = {'growth_rate_real_estate': 0.05,
              'growth_rate_stocks': 0.05,
              'stock_price': 100,
              'currency_exchange_rate': 8.5,
              'mortgage_overpay_month': 5000}

port = utils.Portfolio()
mort = utils.Mortgage(50, 0.0305, 3.6e6, 30, 12)
apartment = utils.RealEstate(4.2e6, mort)
sim = Simulation(scenario_1, apartment, port)

In [4]:
sim.run()

In [5]:
pd.DataFrame(sim.real_estate.history).head()

,mortgage_amount,price_index,current_price,fee,interest_amount,capital_downpayment
0,3.600000e+06,1.000000,4.200000e+06,50,9150.000000,11124.995854
1,3.588875e+06,1.004167,4.217500e+06,50,9121.723969,11153.271885
2,3.577722e+06,1.008351,4.235073e+06,50,9093.376069,11181.619785
3,3.566540e+06,1.012552,4.252719e+06,50,9064.956119,11210.039735
4,3.555330e+06,1.016771,4.270439e+06,50,9036.463935,11238.531919
